# P4 Regularización, Normalización y Aumentado de datos



## Ejercicio:

Probar todas las técnicas presentadas para obtener un acierto en test > 99%. Se aconseja no malgastar datos de entrenamiento y por lo tanto emplear todo el training set para el entrenamiento. No emplear conjunto de validación y emplear el test set al final para calcular el acierto final.

In [2]:
## Solución


# Importar y normalizar datos

from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from keras import Sequential
from keras.layers import Dense, Input, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')



datagen.fit(x_train)

from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input((28,28,1)))
model.add(Reshape((784,)))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2, min_lr=0.00001)

checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=50
batch_size=128
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


training set (60000, 28, 28, 1)
test set (10000, 28, 28, 1)
Epoch 1/50
468/469 [============================>.] - ETA: 0s - loss: 0.6899 - accuracy: 0.8047
Epoch 1: val_accuracy improved from -inf to 0.93600, saving model to best_model.h5
469/469 [==============================] - 31s 61ms/step - loss: 0.6893 - accuracy: 0.8048 - val_loss: 0.1980 - val_accuracy: 0.9360 - lr: 0.0100


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/50
469/469 [==============================] - ETA: 0s - loss: 0.3211 - accuracy: 0.9016
Epoch 2: val_accuracy improved from 0.93600 to 0.94760, saving model to best_model.h5
469/469 [==============================] - 22s 48ms/step - loss: 0.3211 - accuracy: 0.9016 - val_loss: 0.1858 - val_accuracy: 0.9476 - lr: 0.0100
Epoch 3/50
468/469 [============================>.] - ETA: 0s - loss: 0.2462 - accuracy: 0.9234
Epoch 3: val_accuracy improved from 0.94760 to 0.97280, saving model to best_model.h5
469/469 [==============================] - 20s 42ms/step - loss: 0.2461 - accuracy: 0.9234 - val_loss: 0.0835 - val_accuracy: 0.9728 - lr: 0.0100
Epoch 4/50
468/469 [============================>.] - ETA: 0s - loss: 0.2082 - accuracy: 0.9342
Epoch 4: val_accuracy did not improve from 0.97280
469/469 [==============================] - 22s 48ms/step - loss: 0.2082 - accuracy: 0.9341 - val_loss: 0.1297 - val_accuracy: 0.9554 - lr: 0.0100
Epoch 5/50
468/469 [============================>.]